# Pre trained model 
Para clasificar imágenes, uno de los modelos preentrenados más populares y efectivos es ResNet (Residual Networks), específicamente ResNet50, que es una red profunda muy eficiente para tareas de clasificación de imágenes.

In [5]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam

ModuleNotFoundError: No module named 'matplotlib'

 Cargamos ResNet50 sin la capa final de clasificación (include_top=False), ya que vamos a agregar nuestra propia capa de clasificación según nuestro número de clases.

 ResNet50 requiere que las imágenes sean redimensionadas a 224x224 píxeles y que sus valores estén normalizados de acuerdo con los valores que fueron utilizados para entrenar el modelo preentrenado

In [ ]:
# Cargar el modelo preentrenado ResNet50 sin la capa final de clasificación
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Congelar las capas del modelo base

NameError: name 'VGG16' is not defined

Añadimos una capa densa con 3 unidades y la activación softmax para clasificación multiclase.

In [ ]:
# Crear el modelo completo añadiendo capas para la clasificación
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Promedio global de las características
    layers.Dense(1024, activation='relu'),  # Capa densa para aprender representaciones
    layers.Dense(3, activation='softmax')  # 3 clases 
])

In [ ]:
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Usamos image_dataset_from_directory para cargar las imágenes desde directorios

In [ ]:
# Preprocesamiento y carga del dataset (asegúrate de tener imágenes en carpetas de cada clase)
train_dir = 'path/to/train'  # Cambia esto por la ruta a tu carpeta de imágenes de entrenamiento
validation_dir = 'path/to/validation'  # Cambia esto por la ruta a tu carpeta de validación

train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
)

validation_dataset = image_dataset_from_directory(
    validation_dir,
    image_size=(224, 224),
    batch_size=32,
)

Entrenamos el modelo con nuestro conjunto de datos

In [ ]:
model.fit(
    train_dataset,
    epochs=10,  # épocas de entrenamiento
    validation_data=validation_dataset
)

In [ ]:
# Curva de loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Mostrar las gráficas
plt.tight_layout()
plt.show()

Después de entrenar las capas superiores, hacemos "fine-tuning" desbloqueando algunas capas del modelo base y entrenando nuevamente con una tasa de aprendizaje menor.

In [ ]:

# Fine-tuning del modelo, desbloquear algunas capas y reentrenar
base_model.trainable = True
fine_tune_at = 100  # Elige en qué capa comenzará a entrenar

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar nuevamente con las capas desbloqueadas
model.fit(
    train_dataset,
    epochs=5,
    validation_data=validation_dataset
)